# Structured and time series data 

Based on the methodology taken by the third place result in the Rossman Kaggle competition as detailed in Guo/Berkhahn's [Entity Embeddings of Categorical Variables](https://arxiv.org/abs/1604.06737). See fastai rossman.ipynb

The motivation behind exploring this architecture is it's relevance to real-world application. Most data used for decision making day-to-day in industry is structured and/or time-series data. Here we explore the end-to-end process of using neural networks with practical structured data problems.

Here we will use a Root Mean Square Error loss function to be consistent with other forecasting models used for this project

NB this notebook was divided into two parts so that the second part could be more iteratively fine tuned

In this part we pre-process the data


In [4]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [5]:
from fastai.structured import *
from fastai.column_data import *
import feather as ftr
from datetime import timedelta

In [6]:
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'

In [7]:
PATH='../input/merged_data/'

In [8]:
from IPython.display import HTML, display


The following returns summarized aggregate information to each table accross each field.

In [6]:
daily_48hh = pd.read_csv(f'{PATH}daily_all_48hh.csv')

Here we read in only daly data where a household has 48 half hourly measurements

In [7]:
daily_48hh.head()

,Unnamed: 0,LCLid,day,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min
0,1,MAC000118,2011-12-15,0.1140,0.183542,0.641,48,0.156322,8.810,0.061
1,2,MAC000118,2011-12-16,0.1510,0.206750,1.029,48,0.190122,9.924,0.061
2,3,MAC000118,2011-12-17,0.1460,0.218854,0.677,48,0.174688,10.505,0.056
3,4,MAC000118,2011-12-18,0.1820,0.274417,1.012,48,0.235795,13.172,0.061
4,5,MAC000118,2011-12-19,0.1175,0.194750,0.695,48,0.164503,9.348,0.057


In [8]:
daily_48hh.drop(columns=['Unnamed: 0'],inplace=True)

In [9]:
display(DataFrameSummary(daily_48hh).summary())

,LCLid,day,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min
count,NaN,NaN,3.46935e+06,3.46935e+06,3.46935e+06,3.46935e+06,3.46935e+06,3.46935e+06,3.46935e+06
mean,NaN,NaN,0.158577,0.211765,0.836838,48,0.172691,10.1647,0.0592012
std,NaN,NaN,0.169372,0.190229,0.668409,0,0.153189,9.13101,0.0849863
min,NaN,NaN,0,0,0,48,0,0,0
25%,NaN,NaN,0.067,0.0982292,0.348,48,0.0691159,4.715,0.02
50%,NaN,NaN,0.1145,0.163417,0.691,48,0.132795,7.844,0.039
75%,NaN,NaN,0.191,0.262542,1.13,48,0.229319,12.602,0.071
max,NaN,NaN,6.9055,6.92825,10.761,48,3.34731,332.556,6.394
counts,3469352,3469352,3469352,3469352,3469352,3469352,3469352,3469352,3469352
uniques,5560,827,10337,396529,6419,1,3245602,399606,2078


In [10]:
weather = pd.read_csv(f'{PATH}weather_daily_darksky.csv')

In [11]:
weather[['day', 'time']] = weather['sunriseTime'].str.split(' ', n=1, expand=True)

In [12]:
weather.head(n=2)

,temperatureMax,temperatureMaxTime,windBearing,icon,dewPoint,temperatureMinTime,cloudCover,windSpeed,pressure,apparentTemperatureMinTime,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureHighTime,apparentTemperatureLow,apparentTemperatureMax,uvIndex,time,sunsetTime,temperatureLow,temperatureMin,temperatureHigh,sunriseTime,temperatureHighTime,uvIndexTime,summary,temperatureLowTime,apparentTemperatureMin,apparentTemperatureMaxTime,apparentTemperatureLowTime,moonPhase,day
0,11.96,2011-11-11 23:00:00,123,fog,9.40,2011-11-11 07:00:00,0.79,3.88,1016.08,2011-11-11 07:00:00,10.87,rain,3.30,0.95,2011-11-11 19:00:00,10.87,11.96,1.0,07:12:14,2011-11-11 16:19:21,10.87,8.85,10.87,2011-11-11 07:12:14,2011-11-11 19:00:00,2011-11-11 11:00:00,Foggy until afternoon.,2011-11-11 19:00:00,6.48,2011-11-11 23:00:00,2011-11-11 19:00:00,0.52,2011-11-11
1,8.59,2011-12-11 14:00:00,198,partly-cloudy-day,4.49,2011-12-11 01:00:00,0.56,3.94,1007.71,2011-12-11 02:00:00,5.62,rain,12.09,0.88,2011-12-11 19:00:00,-0.64,5.72,1.0,07:57:02,2011-12-11 15:52:53,3.09,2.48,8.59,2011-12-11 07:57:02,2011-12-11 14:00:00,2011-12-11 12:00:00,Partly cloudy throughout the day.,2011-12-12 07:00:00,0.11,2011-12-11 20:00:00,2011-12-12 08:00:00,0.53,2011-12-11


In [13]:
#convert time string to timestamp
weather['time'] = pd.to_timedelta(weather['time'])

Bank Holidays

In [14]:
bank_holidays = pd.read_csv(f'{PATH}uk_bank_holidays.csv')

In [15]:
bank_holidays = bank_holidays.rename(columns={'Bank holidays': 'day'})


In [16]:
bank_holidays.head()

,day,Type
0,2012-12-26,Boxing Day
1,2012-12-25,Christmas Day
2,2012-08-27,Summer bank holiday
3,2012-05-06,Queen?s Diamond Jubilee (extra bank holiday)
4,2012-04-06,Spring bank holiday (substitute day)


In [17]:
acorn_details = pd.read_csv(f'{PATH}acorn_details.csv')

In [18]:
acorn_details.head()

,MAIN CATEGORIES,CATEGORIES,REFERENCE,ACORN-A,ACORN-B,ACORN-C,ACORN-D,ACORN-E,ACORN-F,ACORN-G,ACORN-H,ACORN-I,ACORN-J,ACORN-K,ACORN-L,ACORN-M,ACORN-N,ACORN-O,ACORN-P,ACORN-Q
0,POPULATION,Age,Age 0-4,77.0,83.0,72.0,100.0,120.0,77.0,97.0,97.0,63.0,119.0,67.0,114.0,113.0,89.0,123.0,138.0,133.0
1,POPULATION,Age,Age 5-17,117.0,109.0,87.0,69.0,94.0,95.0,102.0,106.0,67.0,95.0,64.0,108.0,116.0,86.0,89.0,136.0,106.0
2,POPULATION,Age,Age 18-24,64.0,73.0,67.0,107.0,100.0,71.0,83.0,89.0,62.0,104.0,459.0,97.0,96.0,86.0,117.0,109.0,110.0
3,POPULATION,Age,Age 25-34,52.0,63.0,62.0,197.0,151.0,66.0,90.0,88.0,63.0,132.0,145.0,109.0,96.0,90.0,140.0,120.0,120.0
4,POPULATION,Age,Age 35-49,102.0,105.0,91.0,124.0,118.0,93.0,102.0,103.0,76.0,111.0,67.0,99.0,98.0,90.0,102.0,103.0,100.0


In [19]:
informations_households = pd.read_csv(f'{PATH}informations_households.csv')

In [20]:
informations_households.head()

,LCLid,stdorToU,Acorn,Acorn_grouped,file
0,MAC005492,ToU,ACORN-,ACORN-,block_0
1,MAC001074,ToU,ACORN-,ACORN-,block_0
2,MAC000002,Std,ACORN-A,Affluent,block_0
3,MAC003613,Std,ACORN-A,Affluent,block_0
4,MAC003597,Std,ACORN-A,Affluent,block_0


## Data Cleaning / Feature Engineering

In [21]:
len(daily_48hh)

3469352

`join_df` is a function for joining tables on specific fields. By default, we'll be doing a left outer join of `right` on the `left` argument using the given fields for each table.

Pandas does joins using the `merge` method. The `suffixes` argument describes the naming convention for duplicate fields. We've elected to leave the duplicate field names on the left untouched, and append a "\_y" to those on the right.

In [16]:
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", suffix))

Join weather/state names.

In [23]:
df = join_df(daily_48hh, informations_households, "LCLid")

In [24]:
df.head(n=2)

,LCLid,day,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min,stdorToU,Acorn,Acorn_grouped,file
0,MAC000118,2011-12-15,0.114,0.183542,0.641,48,0.156322,8.810,0.061,Std,ACORN-G,Comfortable,block_60
1,MAC000118,2011-12-16,0.151,0.206750,1.029,48,0.190122,9.924,0.061,Std,ACORN-G,Comfortable,block_60


In [25]:
df.drop(columns=['file'],inplace=True)

In [26]:
df = join_df(df, weather, "day")

In [27]:
df = join_df(df, bank_holidays, "day")

In [28]:
df.head(n=2)

,LCLid,day,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min,stdorToU,Acorn,Acorn_grouped,temperatureMax,temperatureMaxTime,windBearing,icon,dewPoint,temperatureMinTime,cloudCover,windSpeed,pressure,apparentTemperatureMinTime,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureHighTime,apparentTemperatureLow,apparentTemperatureMax,uvIndex,time,sunsetTime,temperatureLow,temperatureMin,temperatureHigh,sunriseTime,temperatureHighTime,uvIndexTime,summary,temperatureLowTime,apparentTemperatureMin,apparentTemperatureMaxTime,apparentTemperatureLowTime,moonPhase,Type
0,MAC000118,2011-12-15,0.114,0.183542,0.641,48,0.156322,8.810,0.061,Std,ACORN-G,Comfortable,7.97,2011-12-15 14:00:00,234,wind,2.41,2011-12-15 00:00:00,0.42,4.71,996.75,2011-12-15 00:00:00,4.25,rain,12.79,0.77,2011-12-15 14:00:00,-2.58,4.38,1.0,08:00:46,2011-12-15 15:52:48,1.80,4.08,7.97,2011-12-15 08:00:46,2011-12-15 14:00:00,2011-12-15 11:00:00,Partly cloudy throughout the day and breezy in...,2011-12-16 08:00:00,1.07,2011-12-15 21:00:00,2011-12-16 08:00:00,0.66,NaN
1,MAC000118,2011-12-16,0.151,0.206750,1.029,48,0.190122,9.924,0.061,Std,ACORN-G,Comfortable,4.68,2011-12-16 00:00:00,315,partly-cloudy-day,1.60,2011-12-16 08:00:00,0.70,3.71,988.10,2011-12-16 10:00:00,0.23,rain,10.96,0.88,2011-12-16 15:00:00,-3.56,0.99,1.0,08:01:35,2011-12-16 15:52:56,0.24,1.80,4.53,2011-12-16 08:01:35,2011-12-16 15:00:00,2011-12-16 11:00:00,Mostly cloudy throughout the day.,2011-12-17 08:00:00,-2.65,2011-12-16 00:00:00,2011-12-17 08:00:00,0.70,NaN


In [29]:
df = df.rename(columns={'Type': 'bankHoliday'})

The following extracts particular date fields from a complete datetime for the purpose of constructing categoricals.

You should *always* consider this feature extraction step when working with date-time. Without expanding your date-time into these additional fields, you can't capture any trend/cyclical behavior as a function of time at any of these granularities. We'll add to every table with a date field.

In [30]:
add_datepart(df, "day", drop=False)

In [31]:
df.head(n=2)

,LCLid,day,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min,stdorToU,Acorn,Acorn_grouped,temperatureMax,temperatureMaxTime,windBearing,icon,dewPoint,temperatureMinTime,cloudCover,windSpeed,pressure,apparentTemperatureMinTime,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureHighTime,apparentTemperatureLow,apparentTemperatureMax,uvIndex,time,sunsetTime,temperatureLow,temperatureMin,temperatureHigh,sunriseTime,temperatureHighTime,uvIndexTime,summary,temperatureLowTime,apparentTemperatureMin,apparentTemperatureMaxTime,apparentTemperatureLowTime,moonPhase,bankHoliday,dayYear,dayMonth,dayWeek,dayDay,dayDayofweek,dayDayofyear,dayIs_month_end,dayIs_month_start,dayIs_quarter_end,dayIs_quarter_start,dayIs_year_end,dayIs_year_start,dayElapsed
0,MAC000118,2011-12-15,0.114,0.183542,0.641,48,0.156322,8.810,0.061,Std,ACORN-G,Comfortable,7.97,2011-12-15 14:00:00,234,wind,2.41,2011-12-15 00:00:00,0.42,4.71,996.75,2011-12-15 00:00:00,4.25,rain,12.79,0.77,2011-12-15 14:00:00,-2.58,4.38,1.0,08:00:46,2011-12-15 15:52:48,1.80,4.08,7.97,2011-12-15 08:00:46,2011-12-15 14:00:00,2011-12-15 11:00:00,Partly cloudy throughout the day and breezy in...,2011-12-16 08:00:00,1.07,2011-12-15 21:00:00,2011-12-16 08:00:00,0.66,NaN,2011,12,50,15,3,349,False,False,False,False,False,False,1323907200
1,MAC000118,2011-12-16,0.151,0.206750,1.029,48,0.190122,9.924,0.061,Std,ACORN-G,Comfortable,4.68,2011-12-16 00:00:00,315,partly-cloudy-day,1.60,2011-12-16 08:00:00,0.70,3.71,988.10,2011-12-16 10:00:00,0.23,rain,10.96,0.88,2011-12-16 15:00:00,-3.56,0.99,1.0,08:01:35,2011-12-16 15:52:56,0.24,1.80,4.53,2011-12-16 08:01:35,2011-12-16 15:00:00,2011-12-16 11:00:00,Mostly cloudy throughout the day.,2011-12-17 08:00:00,-2.65,2011-12-16 00:00:00,2011-12-17 08:00:00,0.70,NaN,2011,12,50,16,4,350,False,False,False,False,False,False,1323993600


In [32]:
#convert time strings to timestamp
df['sunsetTime'] = pd.to_datetime(df['sunsetTime'], format = '%Y-%m-%d %H:%M:%S')
df['sunriseTime'] = pd.to_datetime(df['sunriseTime'], format = '%Y-%m-%d %H:%M:%S')

In [33]:
df['daylightMins'] =(df['sunsetTime']-df['sunriseTime']).astype('timedelta64[m]')

In [34]:
df.drop(columns=['time'],inplace=True)
df.head(n=2)

,LCLid,day,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min,stdorToU,Acorn,Acorn_grouped,temperatureMax,temperatureMaxTime,windBearing,icon,dewPoint,temperatureMinTime,cloudCover,windSpeed,pressure,apparentTemperatureMinTime,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureHighTime,apparentTemperatureLow,apparentTemperatureMax,uvIndex,sunsetTime,temperatureLow,temperatureMin,temperatureHigh,sunriseTime,temperatureHighTime,uvIndexTime,summary,temperatureLowTime,apparentTemperatureMin,apparentTemperatureMaxTime,apparentTemperatureLowTime,moonPhase,bankHoliday,dayYear,dayMonth,dayWeek,dayDay,dayDayofweek,dayDayofyear,dayIs_month_end,dayIs_month_start,dayIs_quarter_end,dayIs_quarter_start,dayIs_year_end,dayIs_year_start,dayElapsed,daylightMins
0,MAC000118,2011-12-15,0.114,0.183542,0.641,48,0.156322,8.810,0.061,Std,ACORN-G,Comfortable,7.97,2011-12-15 14:00:00,234,wind,2.41,2011-12-15 00:00:00,0.42,4.71,996.75,2011-12-15 00:00:00,4.25,rain,12.79,0.77,2011-12-15 14:00:00,-2.58,4.38,1.0,2011-12-15 15:52:48,1.80,4.08,7.97,2011-12-15 08:00:46,2011-12-15 14:00:00,2011-12-15 11:00:00,Partly cloudy throughout the day and breezy in...,2011-12-16 08:00:00,1.07,2011-12-15 21:00:00,2011-12-16 08:00:00,0.66,NaN,2011,12,50,15,3,349,False,False,False,False,False,False,1323907200,472.0
1,MAC000118,2011-12-16,0.151,0.206750,1.029,48,0.190122,9.924,0.061,Std,ACORN-G,Comfortable,4.68,2011-12-16 00:00:00,315,partly-cloudy-day,1.60,2011-12-16 08:00:00,0.70,3.71,988.10,2011-12-16 10:00:00,0.23,rain,10.96,0.88,2011-12-16 15:00:00,-3.56,0.99,1.0,2011-12-16 15:52:56,0.24,1.80,4.53,2011-12-16 08:01:35,2011-12-16 15:00:00,2011-12-16 11:00:00,Mostly cloudy throughout the day.,2011-12-17 08:00:00,-2.65,2011-12-16 00:00:00,2011-12-17 08:00:00,0.70,NaN,2011,12,50,16,4,350,False,False,False,False,False,False,1323993600,471.0


Next we'll fill in missing values to avoid complications with `NA`'s. `NA` (not available) is how Pandas indicates missing values; many models have problems when missing values are present, so it's always important to think about how to deal with them. In these cases, we are picking an arbitrary *signal value* that doesn't otherwise appear in the data.

In [35]:
#df['nn'] = df.nn.fillna(1900).astype(np.int32)


## Durations

It is common when working with time series data to extract data that explains relationships across rows as opposed to columns, e.g.:
* Running averages
* Time until next event
* Time since last event

This is often difficult to do with most table manipulation frameworks, since they are designed to work with relationships across columns. As such, we've created a class to handle this type of data.

We'll define a function `get_elapsed` for cumulative counting across a sorted dataframe. Given a particular field `fld` to monitor, this function will start tracking time since the last occurrence of that field. When the field is seen again, the counter is set to zero.

Upon initialization, this will result in datetime na's until the field is encountered. This is reset every time a new store is seen. We'll see how to use this shortly.

In [36]:
def get_elapsed(fld, pre):
    day1 = np.timedelta64(1, 'D')
    print(f'day1: {str(day1)}')
    last_date = np.datetime64()
    last_hh = 0
    res = []

    for s,v,d in zip(df.LCLid.values,df[fld].values, df.day.values):
        if s != last_hh:
            last_date = np.datetime64()
            last_hh = s
        if v: last_date = d
        res.append(((d-last_date).astype('timedelta64[D]') / day1))
    df[pre+fld] = res

We'll be applying this to a subset of columns:

In [37]:
columns = ["day", "LCLid", "bankHoliday"]

Let's walk through an example.

Say we're looking at Bank Holiday. We'll first sort by LCLid, then day, and then call `add_elapsed('BankHoliday', 'After')`:
This will apply to each row with Bank Holiday:
* A applied to every row of the dataframe in order of LCLid and day
* Will add to the dataframe the days since seeing a Bank Holiday
* If we sort in the other direction, this will count the days until another holiday.

In [38]:
fld = 'bankHoliday'
df = df.sort_values(['LCLid', 'day'])
get_elapsed(fld, 'After')
df = df.sort_values(['LCLid', 'day'], ascending=[True, False])
get_elapsed(fld, 'Before')

day1: 1 days
day1: 1 days


In [39]:
df.head(n=2)

,LCLid,day,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min,stdorToU,Acorn,Acorn_grouped,temperatureMax,temperatureMaxTime,windBearing,icon,dewPoint,temperatureMinTime,cloudCover,windSpeed,pressure,apparentTemperatureMinTime,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureHighTime,apparentTemperatureLow,apparentTemperatureMax,uvIndex,sunsetTime,temperatureLow,temperatureMin,temperatureHigh,sunriseTime,temperatureHighTime,uvIndexTime,summary,temperatureLowTime,apparentTemperatureMin,apparentTemperatureMaxTime,apparentTemperatureLowTime,moonPhase,bankHoliday,dayYear,dayMonth,dayWeek,dayDay,dayDayofweek,dayDayofyear,dayIs_month_end,dayIs_month_start,dayIs_quarter_end,dayIs_quarter_start,dayIs_year_end,dayIs_year_start,dayElapsed,daylightMins,AfterbankHoliday,BeforebankHoliday
221349,MAC000002,2014-02-27,0.2180,0.427458,1.350,48,0.406681,20.518,0.080,Std,ACORN-A,Affluent,10.31,2014-02-27 14:00:00,224,partly-cloudy-day,3.08,2014-02-27 23:00:00,0.32,4.14,1007.02,2014-02-27 22:00:00,10.31,rain,12.04,0.74,2014-02-27 14:00:00,0.82,10.31,2.0,2014-02-27 17:37:35,3.43,3.93,10.31,2014-02-27 06:51:45,2014-02-27 14:00:00,2014-02-27 12:00:00,Partly cloudy until evening.,2014-02-28 02:00:00,1.41,2014-02-27 14:00:00,2014-02-28 02:00:00,0.93,NaN,2014,2,9,27,3,58,False,False,False,False,False,False,1393459200,645.0,0.0,0.0
221348,MAC000002,2014-02-26,0.1515,0.256833,1.028,48,0.196095,12.328,0.076,Std,ACORN-A,Affluent,11.29,2014-02-26 13:00:00,227,partly-cloudy-day,2.74,2014-02-26 07:00:00,0.26,3.82,1012.73,2014-02-26 07:00:00,11.29,rain,13.00,0.73,2014-02-26 13:00:00,3.03,11.29,2.0,2014-02-26 17:35:49,6.01,4.17,11.29,2014-02-26 06:53:52,2014-02-26 13:00:00,2014-02-26 12:00:00,Partly cloudy throughout the day.,2014-02-27 00:00:00,1.67,2014-02-26 13:00:00,2014-02-27 00:00:00,0.90,NaN,2014,2,9,26,2,57,False,False,False,False,False,False,1393372800,641.0,0.0,0.0


We'll do this for two more fields.

We're going to set the active index to Date.

In [40]:
df = df.set_index("day")

Then set null values from elapsed field calculations to 0.

In [41]:
columns = ['bankHoliday']

In [42]:
for o in ['Before', 'After']:
    for p in columns:
        a = o+p
        df[a] = df[a].fillna(0).astype(int)

In [43]:
df.head(n=2)

,LCLid,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min,stdorToU,Acorn,Acorn_grouped,temperatureMax,temperatureMaxTime,windBearing,icon,dewPoint,temperatureMinTime,cloudCover,windSpeed,pressure,apparentTemperatureMinTime,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureHighTime,apparentTemperatureLow,apparentTemperatureMax,uvIndex,sunsetTime,temperatureLow,temperatureMin,temperatureHigh,sunriseTime,temperatureHighTime,uvIndexTime,summary,temperatureLowTime,apparentTemperatureMin,apparentTemperatureMaxTime,apparentTemperatureLowTime,moonPhase,bankHoliday,dayYear,dayMonth,dayWeek,dayDay,dayDayofweek,dayDayofyear,dayIs_month_end,dayIs_month_start,dayIs_quarter_end,dayIs_quarter_start,dayIs_year_end,dayIs_year_start,dayElapsed,daylightMins,AfterbankHoliday,BeforebankHoliday
day,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2014-02-27,MAC000002,0.2180,0.427458,1.350,48,0.406681,20.518,0.080,Std,ACORN-A,Affluent,10.31,2014-02-27 14:00:00,224,partly-cloudy-day,3.08,2014-02-27 23:00:00,0.32,4.14,1007.02,2014-02-27 22:00:00,10.31,rain,12.04,0.74,2014-02-27 14:00:00,0.82,10.31,2.0,2014-02-27 17:37:35,3.43,3.93,10.31,2014-02-27 06:51:45,2014-02-27 14:00:00,2014-02-27 12:00:00,Partly cloudy until evening.,2014-02-28 02:00:00,1.41,2014-02-27 14:00:00,2014-02-28 02:00:00,0.93,NaN,2014,2,9,27,3,58,False,False,False,False,False,False,1393459200,645.0,0,0
2014-02-26,MAC000002,0.1515,0.256833,1.028,48,0.196095,12.328,0.076,Std,ACORN-A,Affluent,11.29,2014-02-26 13:00:00,227,partly-cloudy-day,2.74,2014-02-26 07:00:00,0.26,3.82,1012.73,2014-02-26 07:00:00,11.29,rain,13.00,0.73,2014-02-26 13:00:00,3.03,11.29,2.0,2014-02-26 17:35:49,6.01,4.17,11.29,2014-02-26 06:53:52,2014-02-26 13:00:00,2014-02-26 12:00:00,Partly cloudy throughout the day.,2014-02-27 00:00:00,1.67,2014-02-26 13:00:00,2014-02-27 00:00:00,0.90,NaN,2014,2,9,26,2,57,False,False,False,False,False,False,1393372800,641.0,0,0


In [44]:
#replace characters in bank holiday
df['bankHoliday'] = df['bankHoliday'].str.replace('?','')

Next we'll demonstrate window functions in pandas to calculate rolling quantities.

Here we're sorting by date (`sort_index()`) and counting the number of events of interest (`sum()`) defined in `columns` in the following week (`rolling()`), grouped by Store (`groupby()`). We do the same in the opposite direction.

For now lets just create a boolean for holiday or not, can revisit to refine later on

In [45]:
df['bankHoliday'].fillna(False, inplace=True)

In [46]:
mask = df.bankHoliday != False
df.loc[mask, 'bankHoliday'] = True

In [47]:
bwd = df[['LCLid']+columns].sort_index().groupby("LCLid").rolling(7, min_periods=1).sum()

In [48]:
fwd = df[['LCLid']+columns].sort_index(ascending=False
                                      ).groupby("LCLid").rolling(7, min_periods=1).sum()

In [49]:
fwd.head(n=2)

bankHoliday
LCLid     day                    
MAC000002 2014-02-27          0.0
          2014-02-26          0.0

Next we want to drop the LCLid indices grouped together in the window function.

Often in pandas, there is an option to do this in place. This is time and memory efficient when working with large datasets.

In [50]:
#bwd.drop('LCLid',1,inplace=True)
bwd.reset_index(inplace=True)

In [51]:
#fwd.drop('LCLid',1,inplace=True)
fwd.reset_index(inplace=True)

In [52]:
fwd.head(n=2)

,LCLid,day,bankHoliday
0,MAC000002,2014-02-27,0.0
1,MAC000002,2014-02-26,0.0


In [53]:
df.reset_index(inplace=True)

In [54]:
df.head(n=2)

,day,LCLid,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min,stdorToU,Acorn,Acorn_grouped,temperatureMax,temperatureMaxTime,windBearing,icon,dewPoint,temperatureMinTime,cloudCover,windSpeed,pressure,apparentTemperatureMinTime,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureHighTime,apparentTemperatureLow,apparentTemperatureMax,uvIndex,sunsetTime,temperatureLow,temperatureMin,temperatureHigh,sunriseTime,temperatureHighTime,uvIndexTime,summary,temperatureLowTime,apparentTemperatureMin,apparentTemperatureMaxTime,apparentTemperatureLowTime,moonPhase,bankHoliday,dayYear,dayMonth,dayWeek,dayDay,dayDayofweek,dayDayofyear,dayIs_month_end,dayIs_month_start,dayIs_quarter_end,dayIs_quarter_start,dayIs_year_end,dayIs_year_start,dayElapsed,daylightMins,AfterbankHoliday,BeforebankHoliday
0,2014-02-27,MAC000002,0.2180,0.427458,1.350,48,0.406681,20.518,0.080,Std,ACORN-A,Affluent,10.31,2014-02-27 14:00:00,224,partly-cloudy-day,3.08,2014-02-27 23:00:00,0.32,4.14,1007.02,2014-02-27 22:00:00,10.31,rain,12.04,0.74,2014-02-27 14:00:00,0.82,10.31,2.0,2014-02-27 17:37:35,3.43,3.93,10.31,2014-02-27 06:51:45,2014-02-27 14:00:00,2014-02-27 12:00:00,Partly cloudy until evening.,2014-02-28 02:00:00,1.41,2014-02-27 14:00:00,2014-02-28 02:00:00,0.93,False,2014,2,9,27,3,58,False,False,False,False,False,False,1393459200,645.0,0,0
1,2014-02-26,MAC000002,0.1515,0.256833,1.028,48,0.196095,12.328,0.076,Std,ACORN-A,Affluent,11.29,2014-02-26 13:00:00,227,partly-cloudy-day,2.74,2014-02-26 07:00:00,0.26,3.82,1012.73,2014-02-26 07:00:00,11.29,rain,13.00,0.73,2014-02-26 13:00:00,3.03,11.29,2.0,2014-02-26 17:35:49,6.01,4.17,11.29,2014-02-26 06:53:52,2014-02-26 13:00:00,2014-02-26 12:00:00,Partly cloudy throughout the day.,2014-02-27 00:00:00,1.67,2014-02-26 13:00:00,2014-02-27 00:00:00,0.90,False,2014,2,9,26,2,57,False,False,False,False,False,False,1393372800,641.0,0,0


Now we'll merge these values onto the df.

In [55]:
df = df.merge(bwd, 'left', ['day', 'LCLid'], suffixes=['', '_bw'])
df = df.merge(fwd, 'left', ['day', 'LCLid'], suffixes=['', '_fw'])

In [56]:
df.drop(columns,1,inplace=True)

In [57]:
df.head(n=2)

,day,LCLid,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min,stdorToU,Acorn,Acorn_grouped,temperatureMax,temperatureMaxTime,windBearing,icon,dewPoint,temperatureMinTime,cloudCover,windSpeed,pressure,apparentTemperatureMinTime,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureHighTime,apparentTemperatureLow,apparentTemperatureMax,uvIndex,sunsetTime,temperatureLow,temperatureMin,temperatureHigh,sunriseTime,temperatureHighTime,uvIndexTime,summary,temperatureLowTime,apparentTemperatureMin,apparentTemperatureMaxTime,apparentTemperatureLowTime,moonPhase,dayYear,dayMonth,dayWeek,dayDay,dayDayofweek,dayDayofyear,dayIs_month_end,dayIs_month_start,dayIs_quarter_end,dayIs_quarter_start,dayIs_year_end,dayIs_year_start,dayElapsed,daylightMins,AfterbankHoliday,BeforebankHoliday,bankHoliday_bw,bankHoliday_fw
0,2014-02-27,MAC000002,0.2180,0.427458,1.350,48,0.406681,20.518,0.080,Std,ACORN-A,Affluent,10.31,2014-02-27 14:00:00,224,partly-cloudy-day,3.08,2014-02-27 23:00:00,0.32,4.14,1007.02,2014-02-27 22:00:00,10.31,rain,12.04,0.74,2014-02-27 14:00:00,0.82,10.31,2.0,2014-02-27 17:37:35,3.43,3.93,10.31,2014-02-27 06:51:45,2014-02-27 14:00:00,2014-02-27 12:00:00,Partly cloudy until evening.,2014-02-28 02:00:00,1.41,2014-02-27 14:00:00,2014-02-28 02:00:00,0.93,2014,2,9,27,3,58,False,False,False,False,False,False,1393459200,645.0,0,0,0.0,0.0
1,2014-02-26,MAC000002,0.1515,0.256833,1.028,48,0.196095,12.328,0.076,Std,ACORN-A,Affluent,11.29,2014-02-26 13:00:00,227,partly-cloudy-day,2.74,2014-02-26 07:00:00,0.26,3.82,1012.73,2014-02-26 07:00:00,11.29,rain,13.00,0.73,2014-02-26 13:00:00,3.03,11.29,2.0,2014-02-26 17:35:49,6.01,4.17,11.29,2014-02-26 06:53:52,2014-02-26 13:00:00,2014-02-26 12:00:00,Partly cloudy throughout the day.,2014-02-27 00:00:00,1.67,2014-02-26 13:00:00,2014-02-27 00:00:00,0.90,2014,2,9,26,2,57,False,False,False,False,False,False,1393372800,641.0,0,0,0.0,0.0


In [58]:
df.to_feather(f'{PATH}df_daily_cat.feather')

In [59]:
df = ftr.read_dataframe(f'{PATH}df_daily_cat.feather')

In [60]:
df.head(n=2)

,day,LCLid,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min,stdorToU,Acorn,Acorn_grouped,temperatureMax,temperatureMaxTime,windBearing,icon,dewPoint,temperatureMinTime,cloudCover,windSpeed,pressure,apparentTemperatureMinTime,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureHighTime,apparentTemperatureLow,apparentTemperatureMax,uvIndex,sunsetTime,temperatureLow,temperatureMin,temperatureHigh,sunriseTime,temperatureHighTime,uvIndexTime,summary,temperatureLowTime,apparentTemperatureMin,apparentTemperatureMaxTime,apparentTemperatureLowTime,moonPhase,dayYear,dayMonth,dayWeek,dayDay,dayDayofweek,dayDayofyear,dayIs_month_end,dayIs_month_start,dayIs_quarter_end,dayIs_quarter_start,dayIs_year_end,dayIs_year_start,dayElapsed,daylightMins,AfterbankHoliday,BeforebankHoliday,bankHoliday_bw,bankHoliday_fw
0,2014-02-27,MAC000002,0.2180,0.427458,1.350,48,0.406681,20.518,0.080,Std,ACORN-A,Affluent,10.31,2014-02-27 14:00:00,224,partly-cloudy-day,3.08,2014-02-27 23:00:00,0.32,4.14,1007.02,2014-02-27 22:00:00,10.31,rain,12.04,0.74,2014-02-27 14:00:00,0.82,10.31,2.0,2014-02-27 17:37:35,3.43,3.93,10.31,2014-02-27 06:51:45,2014-02-27 14:00:00,2014-02-27 12:00:00,Partly cloudy until evening.,2014-02-28 02:00:00,1.41,2014-02-27 14:00:00,2014-02-28 02:00:00,0.93,2014,2,9,27,3,58,False,False,False,False,False,False,1393459200,645.0,0,0,0.0,0.0
1,2014-02-26,MAC000002,0.1515,0.256833,1.028,48,0.196095,12.328,0.076,Std,ACORN-A,Affluent,11.29,2014-02-26 13:00:00,227,partly-cloudy-day,2.74,2014-02-26 07:00:00,0.26,3.82,1012.73,2014-02-26 07:00:00,11.29,rain,13.00,0.73,2014-02-26 13:00:00,3.03,11.29,2.0,2014-02-26 17:35:49,6.01,4.17,11.29,2014-02-26 06:53:52,2014-02-26 13:00:00,2014-02-26 12:00:00,Partly cloudy throughout the day.,2014-02-27 00:00:00,1.67,2014-02-26 13:00:00,2014-02-27 00:00:00,0.90,2014,2,9,26,2,57,False,False,False,False,False,False,1393372800,641.0,0,0,0.0,0.0


In [61]:
#convert time strings to timestamp
#try with and without these columns - initially will try without
df.drop(columns=['temperatureMaxTime', 'temperatureMinTime',
                'apparentTemperatureMinTime','apparentTemperatureHighTime',
                'sunsetTime','sunriseTime','uvIndexTime',
                'temperatureHighTime','apparentTemperatureMaxTime',
                'apparentTemperatureLowTime','temperatureLowTime'], inplace=True)
'''
df['temperatureMaxTime'] = pd.to_datetime(df['temperatureMaxTime'], format = '%Y-%m-%d %H:%M:%S')
df['temperatureMinTime'] = pd.to_datetime(df['temperatureMinTime'], format = '%Y-%m-%d %H:%M:%S')
df['apparentTemperatureMinTime'] = pd.to_datetime(df['apparentTemperatureMinTime'], format = '%Y-%m-%d %H:%M:%S')
df['apparentTemperatureHighTime'] = pd.to_datetime(df['apparentTemperatureHighTime'], format = '%Y-%m-%d %H:%M:%S')
df['sunsetTime'] = pd.to_datetime(df['sunsetTime'], format = '%Y-%m-%d %H:%M:%S')
df['sunriseTime'] = pd.to_datetime(df['sunriseTime'], format = '%Y-%m-%d %H:%M:%S')
df['uvIndexTime'] = pd.to_datetime(df['uvIndexTime'], format = '%Y-%m-%d %H:%M:%S')
df['temperatureHighTime'] = pd.to_datetime(df['temperatureHighTime'], format = '%Y-%m-%d %H:%M:%S')
df['apparentTemperatureMaxTime'] = pd.to_datetime(df['apparentTemperatureMaxTime'], format = '%Y-%m-%d %H:%M:%S')
df['apparentTemperatureLowTime'] = pd.to_datetime(df['apparentTemperatureLowTime'], format = '%Y-%m-%d %H:%M:%S')
'''


"\ndf['temperatureMaxTime'] = pd.to_datetime(df['temperatureMaxTime'], format = '%Y-%m-%d %H:%M:%S')\ndf['temperatureMinTime'] = pd.to_datetime(df['temperatureMinTime'], format = '%Y-%m-%d %H:%M:%S')\ndf['apparentTemperatureMinTime'] = pd.to_datetime(df['apparentTemperatureMinTime'], format = '%Y-%m-%d %H:%M:%S')\ndf['apparentTemperatureHighTime'] = pd.to_datetime(df['apparentTemperatureHighTime'], format = '%Y-%m-%d %H:%M:%S')\ndf['sunsetTime'] = pd.to_datetime(df['sunsetTime'], format = '%Y-%m-%d %H:%M:%S')\ndf['sunriseTime'] = pd.to_datetime(df['sunriseTime'], format = '%Y-%m-%d %H:%M:%S')\ndf['uvIndexTime'] = pd.to_datetime(df['uvIndexTime'], format = '%Y-%m-%d %H:%M:%S')\ndf['temperatureHighTime'] = pd.to_datetime(df['temperatureHighTime'], format = '%Y-%m-%d %H:%M:%S')\ndf['apparentTemperatureMaxTime'] = pd.to_datetime(df['apparentTemperatureMaxTime'], format = '%Y-%m-%d %H:%M:%S')\ndf['apparentTemperatureLowTime'] = pd.to_datetime(df['apparentTemperatureLowTime'], format = '%Y-%

It's usually a good idea to back up large tables of extracted / wrangled features before you join them onto another one, that way you can go back to it easily if you need to make changes to it.

In [62]:
df.head(n=2)

,day,LCLid,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_sum,energy_min,stdorToU,Acorn,Acorn_grouped,temperatureMax,windBearing,icon,dewPoint,cloudCover,windSpeed,pressure,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureLow,apparentTemperatureMax,uvIndex,temperatureLow,temperatureMin,temperatureHigh,summary,apparentTemperatureMin,moonPhase,dayYear,dayMonth,dayWeek,dayDay,dayDayofweek,dayDayofyear,dayIs_month_end,dayIs_month_start,dayIs_quarter_end,dayIs_quarter_start,dayIs_year_end,dayIs_year_start,dayElapsed,daylightMins,AfterbankHoliday,BeforebankHoliday,bankHoliday_bw,bankHoliday_fw
0,2014-02-27,MAC000002,0.2180,0.427458,1.350,48,0.406681,20.518,0.080,Std,ACORN-A,Affluent,10.31,224,partly-cloudy-day,3.08,0.32,4.14,1007.02,10.31,rain,12.04,0.74,0.82,10.31,2.0,3.43,3.93,10.31,Partly cloudy until evening.,1.41,0.93,2014,2,9,27,3,58,False,False,False,False,False,False,1393459200,645.0,0,0,0.0,0.0
1,2014-02-26,MAC000002,0.1515,0.256833,1.028,48,0.196095,12.328,0.076,Std,ACORN-A,Affluent,11.29,227,partly-cloudy-day,2.74,0.26,3.82,1012.73,11.29,rain,13.00,0.73,3.03,11.29,2.0,6.01,4.17,11.29,Partly cloudy throughout the day.,1.67,0.90,2014,2,9,26,2,57,False,False,False,False,False,False,1393372800,641.0,0,0,0.0,0.0


In [63]:
df.to_feather(f'{PATH}df_daily_cat_no_dates.feather')

See part b for deep learning code

### Create Delayed dataset

Here we offset the energy_sum by 7 days from the rest of the data

We want to revisit this - some data we will have an existing forecast (eg 7 day ahead weather) for so we wouldnt need to backdate it

Also, this doesnt truly relect the proerties of actual day in question (eg day may be bank holiday - but are forcing back a week)

In [9]:
df = ftr.read_dataframe(f'{PATH}df_daily_cat_no_dates.feather')

In [10]:
df["last_week"] = df["day"] + timedelta(days=-7)

In [11]:
df_now = df[["day", "energy_sum"]]

In [12]:
df.drop(columns=['day', 'energy_sum'],inplace=True)

In [13]:
df.rename(columns={'last_week': 'day'}, inplace=True)

In [14]:
df.head(n=2)

,LCLid,energy_median,energy_mean,energy_max,energy_count,energy_std,energy_min,stdorToU,Acorn,Acorn_grouped,temperatureMax,windBearing,icon,dewPoint,cloudCover,windSpeed,pressure,apparentTemperatureHigh,precipType,visibility,humidity,apparentTemperatureLow,apparentTemperatureMax,uvIndex,temperatureLow,temperatureMin,temperatureHigh,summary,apparentTemperatureMin,moonPhase,dayYear,dayMonth,dayWeek,dayDay,dayDayofweek,dayDayofyear,dayIs_month_end,dayIs_month_start,dayIs_quarter_end,dayIs_quarter_start,dayIs_year_end,dayIs_year_start,dayElapsed,daylightMins,AfterbankHoliday,BeforebankHoliday,bankHoliday_bw,bankHoliday_fw,day
0,MAC000002,0.2180,0.427458,1.350,48,0.406681,0.080,Std,ACORN-A,Affluent,10.31,224,partly-cloudy-day,3.08,0.32,4.14,1007.02,10.31,rain,12.04,0.74,0.82,10.31,2.0,3.43,3.93,10.31,Partly cloudy until evening.,1.41,0.93,2014,2,9,27,3,58,False,False,False,False,False,False,1393459200,645.0,0,0,0.0,0.0,2014-02-20
1,MAC000002,0.1515,0.256833,1.028,48,0.196095,0.076,Std,ACORN-A,Affluent,11.29,227,partly-cloudy-day,2.74,0.26,3.82,1012.73,11.29,rain,13.00,0.73,3.03,11.29,2.0,6.01,4.17,11.29,Partly cloudy throughout the day.,1.67,0.90,2014,2,9,26,2,57,False,False,False,False,False,False,1393372800,641.0,0,0,0.0,0.0,2014-02-19


* Very memory intensive - uses >128GB RAM + 400 GB swap *

In [ ]:
df = join_df(df, df_now, "day")

In [ ]:
df.to_feather(f'{PATH}df_week_delayed_daily_cat_no_dates.feather')